In [1]:
import pandas as pd

## Importing Data ##

### Importing Shipped Order Data ###

In [2]:
# Import Shipped Order Data
order_df = pd.read_csv('order_lines.csv', index_col = 0, header = 0, encoding='latin-1')
print('size:',order_df.shape)
order_df.head()

size: (5208, 9)


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros
112,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47
14485,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03
14486,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45
14487,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45
14488,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16


In [3]:
#Check dataframe size
order_df.shape

(5208, 9)

In [4]:
#Check if any missing values
order_df.isnull().sum()

Date                 0
Month-Year           0
Warehouse Code       0
Customer Code        0
Order Number         0
Order Line Number    0
Item Code            0
Units                0
Euros                0
dtype: int64

### Importing Conversion Data ###

In [5]:
conversion_df = pd.read_csv('uom_conversions.csv', index_col = 0, header = 0)
print('size:',conversion_df.shape)
conversion_df.head()

size: (557, 2)


,Item Code,Conversion Ratio
51,995097-97,0.0242
100,995096-96,0.0242
113,8949591-91,0.0474
131,952547-47,0.0368
276,866370-70,0.0400


### Importing Distance Data ###

In [6]:
distance_df = pd.read_csv('distances.csv', index_col = 0, header = 0)
print('size:',distance_df.shape)
distance_df.head()

size: (19, 11)


,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air
19,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0
610,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0
676,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0
682,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.694,0.0,0.0,0.0
1375,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.437,0.0,0.0,0.0


### Importing GPS Data ###

In [7]:
gps_df = pd.read_csv('gps_locations.csv', index_col = 0, header = 0)
print('size:',gps_df.shape)
gps_df.head()

size: (19, 3)


,Location,GPS 1,GPS 2
867,"BULGARIA, DOLNI BOGROV",42.701462,23.490811
1087,"FRANCE, AUBROMETZ",50.303749,2.176058
1153,"FRANCE, BELLEVILLE",48.871184,2.386682
1371,"FRANCE, CLICHY CEDEX",48.904099,2.304123
1534,"FRANCE, GAEL",48.116682,-2.234640


## Processing Data

In order to convert quantities from order table into weights in kg, we have to merch the order line table with the conversion table. After that, we can calculate the total weights in kg by multiplying the number of units by conversion ratio. For this case, we use gross weights of the products which include the packagings to calculate the total weights

In [8]:
#Join the order and conversions table using Item Code
merged_order = order_df.merge(conversion_df, on = 'Item Code', how = 'left') 
merged_order.head()

,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200


In [9]:
#Check data type
merged_order.dtypes

Date                  object
Month-Year            object
Warehouse Code         int64
Customer Code         object
Order Number           int64
Order Line Number      int64
Item Code             object
Units                float64
Euros                float64
Conversion Ratio     float64
dtype: object

Since there is one value misising in the conversion ration column, we will fill that missing value with the median conversion ratio

In [10]:
#check if any misisng value
merged_order.isnull().sum()

Date                 0
Month-Year           0
Warehouse Code       0
Customer Code        0
Order Number         0
Order Line Number    0
Item Code            0
Units                0
Euros                0
Conversion Ratio     1
dtype: int64

In [11]:
#Fill the missing data with the median of conversion rate
merged_order['Conversion Ratio'] = merged_order['Conversion Ratio'].fillna(merged_order['Conversion Ratio'].mean(numeric_only = True))
print(merged_order.count())

Date                 5208
Month-Year           5208
Warehouse Code       5208
Customer Code        5208
Order Number         5208
Order Line Number    5208
Item Code            5208
Units                5208
Euros                5208
Conversion Ratio     5208
dtype: int64


In [12]:
#Calculate total weight in kg
merged_order['Total Weight (kg)'] = merged_order['Units'] * merged_order['Conversion Ratio']
merged_order.head()

,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,Total Weight (kg)
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400,113.168
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200,8.000
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219,11.095
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219,11.095
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200,12.800


In [13]:
#Join the distances with gps data
distance_df['Location'] = distance_df['Customer Country'] + ', ' + distance_df['Customer City']
merged_distance = pd.merge(distance_df, gps_df, on = 'Location', how = 'left')
merged_distance.head()

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950
1,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
2,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF",52.304594,10.312521
3,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.694,0.0,0.0,0.0,"FRANCE, BELLEVILLE",48.871184,2.386682
4,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.437,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL",48.624666,2.595756


In [14]:
#Join all the tables together
merged_df = pd.merge(merged_order, merged_distance, on = ['Warehouse Code', 'Customer Code'], how = 'left')
print('size:',merged_df.shape)
merged_df

size: (5208, 23)


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,...,Warehouse City,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400,...,CHALONS-EN-CHAMPAGNE,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5203,2021-10-28 00:00:00.000,10-2021,3403434,5397843-43,21089497,84,802065-65,80.0,72.96,0.02200,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
5204,2021-10-28 00:00:00.000,10-2021,3403434,5397843-43,21089497,85,942082-82,200.0,391.88,0.03200,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
5205,2021-10-28 00:00:00.000,10-2021,3403434,5397843-43,21089497,86,8022922-22,400.0,581.40,0.03200,...,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
5206,2021-11-01 00:00:00.000,10-2021,3403434,5340669-69,21004473,1,982884-84,2000.0,2980.00,0.14600,...,CHALONS-EN-CHAMPAGNE,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF",52.304594,10.312521


## Calculation ##

### Calculate total CO2 at order line level

The total amount of carbon from downstream transport will be calculated based on the GHG protocol:
    
**∑ (quantity of goods sold (tonnes) × distance travelled in transport legs (km) × emission factor of transport mode or vehicle type (kg CO2e/tonne-km))**

From the dataset, we know that the emission factors (CO2e) for Road, Rail, Sea, Air are 0.096, 0.028, 0.01, 2.1 (kg CO2e/tonne-km) respectively. 

In [21]:
groupby_order = ['Date', 'Month-Year', 'Warehouse Code', 'Warehouse Name', 'Warehouse Country', 'Warehouse City','Item Code',
              'Customer Code', 'Customer Country', 'Customer City', 'Location', 'GPS 1', 'GPS 2', 
              'Road', 'Rail', 'Sea', 'Air', 'Order Number']

# Grouping by the specified columns and summing the 'Units' and 'KG' columns
merged_df = merged_df.groupby(groupby_order)[['Units', 'Total Weight (kg)']].sum().reset_index()
merged_df.head()


,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Item Code,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,Total Weight (kg)
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,963543-43,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,2.0,113.168
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,802077-77,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,300.0,5.700
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,8022921-21,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,400.0,12.800
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,8128704-04,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,400.0,8.368
4,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,8308591-91,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,500.0,11.095


In [22]:
co2_e = {'Road': 0.096, 'Rail': 0.028, 'Sea': 0.01, 'Air': 2.1}

#Calculate total CO2
for mode in (co2_e.keys()):
    merged_df['CO2 ' + mode] = co2_e[mode] * merged_df['Total Weight (kg)'].astype(float)/1000 * merged_df[mode].astype(float)
merged_df['Total CO2'] = merged_df[['CO2 ' + mode for mode in co2_e.keys()]].sum(axis = 1)
merged_df


,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Item Code,Customer Code,Customer Country,Customer City,...,Sea,Air,Order Number,Units,Total Weight (kg),CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,Total CO2
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,963543-43,5002915-15,FRANCE,LES ANGLES,...,0.0,0.0,20247100,2.0,113.1680,8.318967,0.0,0.0,0.0,8.318967
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,802077-77,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,20258239,300.0,5.7000,0.154030,0.0,0.0,0.0,0.154030
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,8022921-21,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,20258239,400.0,12.8000,0.345892,0.0,0.0,0.0,0.345892
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,8128704-04,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,20258239,400.0,8.3680,0.226127,0.0,0.0,0.0,0.226127
4,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,8308591-91,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,20258239,500.0,11.0950,0.299818,0.0,0.0,0.0,0.299818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5119,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,897001-01,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,21089497,125.0,5.3800,0.145383,0.0,0.0,0.0,0.145383
5120,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,919289-89,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,21089497,80.0,1.3688,0.036989,0.0,0.0,0.0,0.036989
5121,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,942082-82,5397843-43,FRANCE,SUCY EN BRIE,...,0.0,0.0,21089497,200.0,6.4000,0.172946,0.0,0.0,0.0,0.172946
5122,2021-11-01 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,982884-84,5340669-69,GERMANY,PEINE - WOLTORF,...,0.0,0.0,21004473,2000.0,292.0000,23.995392,0.0,0.0,0.0,23.995392


In [18]:
merged_df.to_csv('Final_report.csv')